In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_VISIBLE_DEVICES=5


In [2]:
import tqdm
import os
import sys
import yaml
import time
import random
import itertools
from omegaconf import OmegaConf
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset, DatasetDict

sys.path.append(os.path.abspath('/homes/80/anya/Documents/llm_tiny_ideas/super-tiny-lms-outer/super-tiny-lms'))


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

/homes/80/anya/anaconda3/envs/coconut-1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [3]:
model_name = "Qwen/Qwen2.5-0.5b"
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
dataset = DatasetDict.load_from_disk(f"../data/my_data/gsm8k")["train"]

In [11]:
[tokenizer.decode(id) for id in tokenizer.encode("Answer: -")]

['Answer', ':', ' -']

In [50]:
tokenized_answers = [tokenizer.encode(answer) for answer in dataset["answer"]]

In [51]:
tokenized_lengths = np.array([len(answer) for answer in tokenized_answers])
is_digits = np.array([answer.isdigit() for answer in dataset["answer"]])

In [52]:
tokenized_lengths.mean(), tokenized_lengths.min(), tokenized_lengths.max(), is_digits.mean()

(np.float64(2.318747490967483),
 np.int64(1),
 np.int64(11),
 np.float64(0.9890271644587181))

In [53]:
for ans, is_digit, question, length in zip(dataset["answer"], is_digits, dataset["question"], tokenized_lengths):
    if is_digit and length > 3:
        print(question, ans, "\n")

James creates a media empire.  He creates a movie for $2000.  Each DVD cost $6 to make.  He sells it for 2.5 times that much.  He sells 500 movies a day for 5 days a week.  How much profit does he make in 20 weeks? Let's think step by step, concisely, and then output the final answer after "Answer:". 448000 

Each bird eats 12 beetles per day, each snake eats 3 birds per day, and each jaguar eats 5 snakes per day. If there are 6 jaguars in a forest, how many beetles are eaten each day? Let's think step by step, concisely, and then output the final answer after "Answer:". 1080 

Mrs. Snyder used to spend 40% of her monthly income on rent and utilities. Her salary was recently increased by $600 so now her rent and utilities only amount to 25% of her monthly income. How much was her previous monthly income? Let's think step by step, concisely, and then output the final answer after "Answer:". 1000 

Noah is a painter. He paints pictures and sells them at the park. He charges $60 for a lar

In [56]:
sum([not isdigit for isdigit, length in zip(is_digits, tokenized_lengths)])

82

In [46]:
tokenized_answers = [tokenizer.encode(answer) for answer, isdigit in zip(dataset["answer"], is_digits) if isdigit]
tokenized_lengths = np.array([len(answer) for answer in tokenized_answers])
tokenized_lengths.mean(), tokenized_lengths.min(), tokenized_lengths.max()

(np.float64(2.2789879583276957), np.int64(1), np.int64(9))

In [24]:
ints = [str(i) for i in range(1000)]
tokenized_ints = [tokenizer.encode(i) for i in ints]
tokenized_ints_lengths = np.array([len(i) for i in tokenized_ints])
tokenized_ints_lengths.mean(), tokenized_ints_lengths.min(), tokenized_ints_lengths.max()
tokenized_ints_set = set(itertools.chain(*tokenized_ints))
np.array([t[-1] in tokenized_ints_set for t in tokenized_answers]).mean(), len(tokenized_ints_set)

(np.float64(1.0), 10)

In [39]:
tokenizer.vocab.keys()
joined = "".join(tokenizer.vocab.keys())
[joined.count(i) for i in "0123456789"]
tokenizer.encode("0123456789")
for token in tokenizer.vocab.keys():
    if token.endswith("-") and (len(token) == 1 or token[-2] != "-"):
        print(token)

=-
][-
((-
":[-
))-
]-
[-
Ġ"-
<-
,-
?-
~-~-
_-
}-
:"-
','-
)[-
Ġ+/-
=-=-=-=-
>-
*-
=(-
\-
Ġ<-
Ġ+-
:-
])-
.-
'-
([-
Ġ//-
Ġ'-
+-+-+-+-+-+-+-+-
/-
=-=-=-=-=-=-=-=-
):-
+-
Ġ;-
Ġ#-
Ġ-*-
)-
==-
()-
('-
~-
Ġ("-
^{-
-
Ġ|-
(-
"-
Ġ{-
("-
!=-
^-
|-
='-
//-
Ġ`-
":-
('/')[-
]=-
",-
Ġ-
Ġ:-
="-
Ġ,-
],[-
-*-
%-
*(-
;-
{-
Ġ['-
=-=-
~-~-~-~-
::-
[::-
+-+-
+-+-+-+-
Ġ(-
Ġ[-
Ġ%-
Ġ<<-
[:-


In [40]:
start_end_idxs = torch.tensor([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
torch.nonzero(start_end_idxs == 1, as_tuple=True)

(tensor([0, 1, 1, 2]), tensor([1, 0, 2, 1]))

In [44]:
def get_last_one_index(x):
    # Find all indices where x equals 1
    indices = (x == 1).nonzero(as_tuple=True)
    print(indices)
    indices = indices[0]
    if indices.numel() == 0:
        return None  # Return None if there's no 1 in the tensor
    # Return the last index
    return indices[-1].item()

# Example usage:
x = torch.tensor([0, 1, 0, 1, 0, 1, 0])
last_index = get_last_one_index(x)
print("Last index of 1:", last_index)

(tensor([1, 3, 5]),)
Last index of 1: 5


# Testing

In [14]:
dummy_model = AutoModelForCausalLM.from_pretrained(config.base_model).to(device)
dummy_ref_model = AutoModelForCausalLM.from_pretrained(config.base_model).to(device)
dummy_tokenizer = AutoTokenizer.from_pretrained(config.base_model)
dummy_dataset = Dataset.load_from_disk(f"../data/my_data/gsm8k/test")
dummy_loader = dataset_loader(dummy_dataset, per_device_batch_size=2, rank=0, world_size=1, seed=0)
dummy_batch = next(dummy_loader)
dummy_questions_text = dummy_batch["question"]
dummy_answers_text = dummy_batch["answer"]
dummy_questions_inputs = dummy_tokenizer(dummy_questions_text, return_tensors="pt", padding=True, padding_side="left")
dummy_answers_inputs = dummy_tokenizer(dummy_answers_text, return_tensors="pt", padding=True)
dummy_questions_inputs = {k: v.to(device) for k, v in dummy_questions_inputs.items()}
dummy_answers_inputs = {k: v.to(device) for k, v in dummy_answers_inputs.items()}
dummy_ref_model.eval()
dummy_model.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [19]:
x, generations, metrics, past_key_values = batch_generate_rnn(
    model=dummy_model,
    ref_model=dummy_ref_model,
    questions_inputs=dummy_questions_inputs,
    answers_inputs=dummy_answers_inputs,
    max_steps=30,
    step_for_answer=20,
    temperature=0.0,
    top_k=None,
    as_full_distribution=False,
    dot_by_dot=False,
    dot_by_dot_id=dummy_tokenizer.encode("....")[0],
    inject_answer_prompt=False,
    answer_prompt_ids=dummy_tokenizer.encode("...Answer:"),
)
print(x[0].shape, generations.shape, metrics)
dummy_tokenizer.batch_decode(generations)

torch.Size([2]) torch.Size([2, 31]) {'answer_logps': 13.639263153076172, 'answer_logps_ref': 13.970451354980469, 'answer_logps_diff': -0.3311886787414551, 'answer_perplexity': 0.002386221196502447, 'answer_perplexity_ref': 0.0011142903240397573, 'answer_perplexity_diff': 0.0012719309888780117, 'logps': 23.325300216674805, 'logps_ref': 20.66714096069336, 'logps_diff': 2.6581602096557617, 'entropy': 0.939110279083252, 'entropy_std': 0.0955267995595932}


[" First, we need to calculate the total number of eggs laid by the ducks in a day. Since Janet's ducks lay 16 eggs per day,",
 ' To find the total number of bolts needed, we need to calculate the number of bolts required for blue and white fibers separately and then add them together.\n\n1']

In [18]:
gen_out = dummy_model.generate(
    input_ids=dummy_questions_inputs["input_ids"],
    attention_mask=dummy_questions_inputs["attention_mask"],
    max_new_tokens=30,
    temperature=1.0,
    top_k=None,
    do_sample=False,
    return_dict_in_generate=True,
)
prompt_length = dummy_questions_inputs["input_ids"].shape[1]
generations = gen_out.sequences if isinstance(gen_out, dict) else gen_out
generations = generations[:, prompt_length:]
print(dummy_tokenizer.batch_decode(generations))


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


[" First, we need to calculate the total number of eggs laid by the ducks in a day. Since Janet's ducks lay 16 eggs per day", ' To find the total number of bolts needed, we need to calculate the number of bolts required for blue and white fibers separately and then add them together.\n\n']


In [20]:
for kv1, kv2 in zip(past_key_values, gen_out.past_key_values):
    for i in range(min(kv1[0].shape[2], kv2[0].shape[2])):
        print(f"{i=}: keys: {kv1[0].shape}, {kv2[0].shape}")
        print(f"{i=}: keys: {kv1[0][0, 0, i, :5].tolist()}")
        print(f"{i=}: keys: {kv2[0][0, 0, i, :5].tolist()}")
        print(f"{i=}: values: {kv1[1].shape}, {kv2[1].shape}")
        print(f"{i=}: values: {kv1[1][0, 0, i, :5].tolist()}")
        print(f"{i=}: values: {kv2[1][0, 0, i, :5].tolist()}")
    break

i=0: keys: torch.Size([2, 2, 116, 64]), torch.Size([2, 2, 115, 64])
i=0: keys: [-8.410331726074219, -3.3071095943450928, -6.438131809234619, 0.6713922619819641, -0.20497480034828186]
i=0: keys: [-8.410331726074219, -3.3071095943450928, -6.438131809234619, 0.6713922619819641, -0.20497480034828186]
i=0: values: torch.Size([2, 2, 116, 64]), torch.Size([2, 2, 115, 64])
i=0: values: [-0.01414407417178154, 0.03389165550470352, -0.02601637691259384, 0.013034755364060402, -0.011828195303678513]
i=0: values: [-0.01414407417178154, 0.03389165550470352, -0.02601637691259384, 0.013034755364060402, -0.011828195303678513]
i=1: keys: torch.Size([2, 2, 116, 64]), torch.Size([2, 2, 115, 64])
i=1: keys: [-9.862237930297852, -7.930609703063965, -7.341488361358643, 2.996467113494873, -0.14621633291244507]
i=1: keys: [-9.862237930297852, -7.930609703063965, -7.341488361358643, 2.996467113494873, -0.14621633291244507]
i=1: values: torch.Size([2, 2, 116, 64]), torch.Size([2, 2, 115, 64])
i=1: values: [0.0100

In [86]:
for k in dummy_model(**dummy_questions_inputs, return_dict=True).keys():
    print(k)

logits
past_key_values
